In [1]:
import pandas as pd
import numpy as np
import re
import os

In [14]:
change_words = pd.read_csv('바이그램트라이그램단어.csv')
change_words.columns = ['바꿀단어','찾을단어']

In [15]:
change_words

,바꿀단어,찾을단어
0,교수님,"교수, 님"
1,기말,"기, 말"
2,비대면,"비대, 면"
3,팀플,"팀, 플"
4,개인적,"개인, 적"
...,...,...
67,비대면강의,"비대, 면, 강의"
68,한학기,"한, 학기"
69,매주과제,"매주, 과제"
70,다른과목,"다른, 과목"


In [9]:
df = pd.read_csv('everytime_review_data_0227.csv')
df

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이,리뷰길이,리뷰_형태소_tuple,리뷰_형태소_list,리뷰_형태소_only_N
0,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13,28,"[('첫', 'Noun'), ('kcu', 'Alpha'), ('강의', 'Noun...","['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에', '이쁠', ...","['첫', '강의', '퀄리티', '쉬움']"
1,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,22년 2학기 수강자,4,0,인문,13,126,"[('그저', 'Adverb'), ('그래요', 'Adjective'), ('다른건...","['그저', '그래요', '다른건', '다', '낫', '배드', '했는데', '기...","['낫', '배드', '기말고사', '서술', '문제', '비중', '제일', '점..."
2,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,22년 2학기 수강자,4,0,인문,13,226,"[('기', 'Modifier'), ('창', 'Noun'), ('강의', 'Nou...","['기', '창', '강의', '평', '피드백', '없고', '강의', '안도',...","['창', '강의', '평', '피드백', '강의', '안도', '꿀강', '퀴즈'..."
3,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...,22년 2학기 수강자,3,0,인문,13,96,"[('과제', 'Noun'), ('하나', 'Noun'), ('도', 'Josa')...","['과제', '하나', '도', '없고', '출석', '퀴즈', '2', '번', ...","['과제', '하나', '출석', '퀴즈', '번', '말', '번만', '퀴즈',..."
4,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,KCU 5과목 정도 수강했는데 그중에 제일 별로 퀴즈2번 기말고사1번 수업에서 안 ...,22년 2학기 수강자,3,0,인문,13,165,"[('KCU', 'Alpha'), ('5', 'Number'), ('과목', 'No...","['KCU', '5', '과목', '정도', '수강', '했는데', '그', '중'...","['과목', '정도', '수강', '중', '제일', '별로', '퀴즈', '번',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44840,영어회화,3,실습,x,2학년,전공,영문,리우빅터,회화 강사 그 이하 밑에 말하는 게 모두 사실 정말 영어과목 줄이던지 제발 영어 따...,19년 2학기 수강자,1,3,자연,4,119,"[('회화', 'Noun'), ('강사', 'Noun'), ('그', 'Noun')...","['회화', '강사', '그', '이하', '밑', '에', '말', '하는', '...","['회화', '강사', '그', '이하', '밑', '말', '게', '모두', '..."
44841,영어회화,3,실습,x,2학년,전공,영문,리우빅터,c플 나왔습니다 시험이 무슨 2주마다 한번 있는데 준비를 해가나 안해가나 점수는 크...,19년 2학기 수강자,1,6,자연,4,169,"[('c', 'Alpha'), ('플', 'Exclamation'), ('나왔습니다...","['c', '플', '나왔습니다', '시험', '이', '무슨', '2', '주마'...","['시험', '무슨', '주마', '한번', '준비', '해', '가나', '점수'..."
44842,영어회화,3,실습,x,2학년,전공,영문,리우빅터,티쳐 2 그룹2 파트너2 안그래도 다른 과제 때문에 바쁜데 1학점짜리가 제일 바쁨 ...,19년 2학기 수강자,1,3,자연,4,167,"[('티', 'Noun'), ('쳐', 'Verb'), ('2', 'Number')...","['티', '쳐', '2', '그룹', '2', '파트너', '2', '안', '그...","['티', '그룹', '파트너', '다른', '과제', '때문', '학점', '리가..."
44843,영어회화,3,실습,x,2학년,전공,영문,리우빅터,19년 2학기 들은사람으로서 한마디 하는데 시험은 총6번입니다 티쳐챗2회 그룹챗2회...,19년 2학기 수강자,1,1,자연,4,262,"[('19년', 'Number'), ('2', 'Number'), ('학기', 'N...","['19년', '2', '학기', '들은', '사람', '으로서', '한마디', '...","['학기', '사람', '한마디', '시험', '총', '번', '티', '챗', ..."


In [10]:
reviews = df[['리뷰','리뷰_형태소_list']].rename(columns={'리뷰_형태소_list': '형태소'})
reviews['형태소'] = reviews['형태소'].apply(lambda x: re.sub(r"\'", '', x[1:-1]))
reviews.head(3)

,리뷰,형태소
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움"
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형, ..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,"기, 창, 강의, 평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한,..."


In [11]:
def NGram(N, DF, ColName, NewColName):
    DF[NewColName] = ''
    for idx in range(len(DF)):
        ColValList = DF._get_value(idx, ColName).split(', ')
        result_list = []
        for val_idx in range(len(ColValList)-(N-1)):
            temp_val = ''.join(ColValList[val_idx:val_idx+N])
            result_list.append(temp_val)
        DF._set_value(idx, NewColName, ', '.join(result_list))
        
NGram(2, reviews, '형태소', '형태소_bi')
NGram(3, reviews, '형태소', '형태소_tri')

In [13]:
reviews.head()

,리뷰,형태소,형태소_bi,형태소_tri
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움","첫kcu, kcu강의, 강의퀄리티, 퀄리티떨어지지만, 떨어지지만에, 에이쁠, 이쁠받...","첫kcu강의, kcu강의퀄리티, 강의퀄리티떨어지지만, 퀄리티떨어지지만에, 떨어지지만..."
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형, ...","그저그래요, 그래요다른건, 다른건다, 다낫, 낫배드, 배드했는데, 했는데기말고사, ...","그저그래요다른건, 그래요다른건다, 다른건다낫, 다낫배드, 낫배드했는데, 배드했는데기..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,"기, 창, 강의, 평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한,...","기창, 창강의, 강의평, 평피드백, 피드백없고, 없고강의, 강의안도, 안도없지만, ...","기창강의, 창강의평, 강의평피드백, 평피드백없고, 피드백없고강의, 없고강의안도, 강..."
3,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...,"과제, 하나, 도, 없고, 출석, 퀴즈, 2, 번, 기, 말, 1, 번만, 하면, ...","과제하나, 하나도, 도없고, 없고출석, 출석퀴즈, 퀴즈2, 2번, 번기, 기말, 말...","과제하나도, 하나도없고, 도없고출석, 없고출석퀴즈, 출석퀴즈2, 퀴즈2번, 2번기,..."
4,KCU 5과목 정도 수강했는데 그중에 제일 별로 퀴즈2번 기말고사1번 수업에서 안 ...,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ...","KCU5, 5과목, 과목정도, 정도수강, 수강했는데, 했는데그, 그중, 중에, 에제...","KCU5과목, 5과목정도, 과목정도수강, 정도수강했는데, 수강했는데그, 했는데그중,..."


In [19]:
temp_df = reviews[['형태소']]
temp_df

,형태소
0,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움"
1,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형, ..."
2,"기, 창, 강의, 평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한,..."
3,"과제, 하나, 도, 없고, 출석, 퀴즈, 2, 번, 기, 말, 1, 번만, 하면, ..."
4,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."
...,...
44840,"회화, 강사, 그, 이하, 밑, 에, 말, 하는, 게, 모두, 사실, 정말, 영어,..."
44841,"c, 플, 나왔습니다, 시험, 이, 무슨, 2, 주마, 다, 한번, 있는데, 준비,..."
44842,"티, 쳐, 2, 그룹, 2, 파트너, 2, 안, 그래도, 다른, 과제, 때문, 에,..."
44843,"19년, 2, 학기, 들은, 사람, 으로서, 한마디, 하는데, 시험, 은, 총, 6..."


In [21]:
bitri = change_words
bitri.head()

,바꿀단어,찾을단어
0,교수님,"교수, 님"
1,기말,"기, 말"
2,비대면,"비대, 면"
3,팀플,"팀, 플"
4,개인적,"개인, 적"


In [22]:
def Change(Target_df, Col_name, Change_df, Find_col, Change_col):
    for idx in range(len(Target_df)):
        target_str = Target_df._get_value(idx, Col_name)
        for change_idx in range(len(Change_df)):
            find_str = Change_df._get_value(change_idx, Find_col)
            change_str = Change_df._get_value(change_idx, Change_col)
            target_str = re.sub(find_str, change_str, target_str)
        Target_df._set_value(idx, Col_name, target_str)
    return Target_df

temp_df = Change(temp_df, '형태소', bitri, '찾을단어', '바꿀단어')
temp_df

,형태소
0,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움"
1,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술형, 1,..."
2,"기창, 강의평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한, 꿀강,..."
3,"과제, 하나, 도, 없고, 출석, 퀴즈, 2번, 기말, 1번만, 하면, 됨, 퀴즈,..."
4,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."
...,...
44840,"회화, 강사, 그, 이하, 밑, 에, 말, 하는, 게, 모두, 사실, 정말, 영어,..."
44841,"c, 플, 나왔습니다, 시험, 이, 무슨, 2, 주마, 다, 한번, 있는데, 준비,..."
44842,"티, 쳐, 2, 그룹, 2, 파트너, 2, 안, 그래도, 다른, 과제, 때문, 에,..."
44843,"19년, 2학기, 들은, 사람, 으로서, 한마디, 하는데, 시험, 은, 총, 6, ..."


In [107]:
temp_df[temp_df['형태소'].str.contains('조별')]

,형태소
250,"기초, 영어, 교수님, 들, 평가, 가, 다, 들, 좋길래, 그냥, 선택, 해서, ..."
283,"일단, 수업, 을, 떠나서, 교수님, 최고, 십, 니, 다, 과제, 라, 해봤자, ..."
285,"교수님, 이, 다정하, 심, 기초, 영어, 라, 그런지, 정말, 쉽게, 잘, 가르쳐..."
345,"일단, 3일, 에, 한, 번꼴, 로, 자료, 조사, 과제, 논문, 이나, 믿을, 수..."
694,"작, 문, 강의, 답, 게, 작문, 과제, 가, 많지만, 하는만큼, 작, 문, 실력..."
...,...
44432,"취업, 준비, 에, 분명히, 도움, 되는, 과목, 인거, 같아요, 그렇지만, N, ..."
44706,"조별, 본문, 읽고, 해석, 문법, 문제풀이, 하, 는, 발표, 1번, 5, 점, ..."
44817,"개인적, 으로, 괜찮은, 수업, 이나, 교양, 패논패, 과목, 특성, 상, 조별, ..."
44819,"조별, 다, 같이, 하기, 싫어서, 팀, 원, 들이, 대충, 따로, 분담, 하면, ..."


In [117]:
rechange = pd.DataFrame({'바꿀단어' : ['중간대체과제','중간고사대체과제','시험2번','과제2번','팀플발표','조별과제','기말과제','기말시험','기말대체','기말레포트'
                          , '조별발표','조별활동','공연채플','중간기말','비대면수업','비대면강의'],
             '찾을단어' : ['중간, 대체과제','중간고사, 대체과제','시험, 2번','과제, 2번','팀플, 발표','조별, 과제'
                       ,'기말, 과제','기말, 시험','기말, 대체','기말, 레포트','조별, 발표','조별, 활동','공연, 채플','중간, 기말','비대면, 수업','비대면, 강의']})

In [118]:
rechange

,바꿀단어,찾을단어
0,중간대체과제,"중간, 대체과제"
1,중간고사대체과제,"중간고사, 대체과제"
2,시험2번,"시험, 2번"
3,과제2번,"과제, 2번"
4,팀플발표,"팀플, 발표"
5,조별과제,"조별, 과제"
6,기말과제,"기말, 과제"
7,기말시험,"기말, 시험"
8,기말대체,"기말, 대체"
9,기말레포트,"기말, 레포트"


In [121]:
temp_df2 = Change(temp_df, '형태소', rechange, '찾을단어', '바꿀단어')
temp_df2

,형태소
0,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움"
1,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술형, 1,..."
2,"기창, 강의평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한, 꿀강,..."
3,"과제, 하나, 도, 없고, 출석, 퀴즈, 2번, 기말, 1번만, 하면, 됨, 퀴즈,..."
4,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."
...,...
44840,"회화, 강사, 그, 이하, 밑, 에, 말, 하는, 게, 모두, 사실, 정말, 영어,..."
44841,"c, 플, 나왔습니다, 시험, 이, 무슨, 2, 주마, 다, 한번, 있는데, 준비,..."
44842,"티, 쳐, 2, 그룹, 2, 파트너, 2, 안, 그래도, 다른, 과제, 때문, 에,..."
44843,"19년, 2학기, 들은, 사람, 으로서, 한마디, 하는데, 시험, 은, 총, 6, ..."


In [122]:
temp_df2[temp_df2['형태소'].str.contains('조별발표')]

,형태소
1632,"한주, 영화, 보고, 쪽글, 제출, 한, 다음, 에, 그, 다음주, 는, 쪽글, 이..."
1671,"필수, 과목, 아니었으면, 안, 들었을, 수업, 일주일, 에, 두번, 수업, 중, ..."
2368,"과제, 대신, 조별발표, 있어요, 수업, 은, 어렵지만, 교수님, 좋아요"
2882,"한승주, 교수님, 수업, 들으신, 분, 들, 은, 교수님, 스타일, 알, 거, 라고..."
3421,"코로나, 로, 인해, 비대면강의, 수강, 매주, 주시, 는, 과제, 때문, 에, 힘..."
...,...
44031,"패논패, 조별발표, 한, 번, 있음, 돈, 받음, 개꿀, 교수님, 상담, 한, 번,..."
44051,"김바, 로미, 교수님, 늘, 에너지, 넘치시고, 상냥하시고, 학생, 입장, 에서, ..."
44322,"지, 혼자, 주저리주저리, 사투리, 는, 알아듣지도, 못, 하겠음, 성적, 인, 영..."
44430,"도움, 된, 다해, 서, 듣고있는데, 그닥, 조별로, 앉아야되, 고, 조별발표, 도..."


In [123]:
temp_df2

,형태소
0,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움"
1,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술형, 1,..."
2,"기창, 강의평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한, 꿀강,..."
3,"과제, 하나, 도, 없고, 출석, 퀴즈, 2번, 기말, 1번만, 하면, 됨, 퀴즈,..."
4,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."
...,...
44840,"회화, 강사, 그, 이하, 밑, 에, 말, 하는, 게, 모두, 사실, 정말, 영어,..."
44841,"c, 플, 나왔습니다, 시험, 이, 무슨, 2, 주마, 다, 한번, 있는데, 준비,..."
44842,"티, 쳐, 2, 그룹, 2, 파트너, 2, 안, 그래도, 다른, 과제, 때문, 에,..."
44843,"19년, 2학기, 들은, 사람, 으로서, 한마디, 하는데, 시험, 은, 총, 6, ..."


In [124]:
df['개정형태소'] = temp_df2['형태소']

In [125]:
df

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이,리뷰길이,리뷰_형태소_tuple,리뷰_형태소_list,리뷰_형태소_only_N,개정형태소
0,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13,28,"[('첫', 'Noun'), ('kcu', 'Alpha'), ('강의', 'Noun...","['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에', '이쁠', ...","['첫', '강의', '퀄리티', '쉬움']","첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움"
1,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,22년 2학기 수강자,4,0,인문,13,126,"[('그저', 'Adverb'), ('그래요', 'Adjective'), ('다른건...","['그저', '그래요', '다른건', '다', '낫', '배드', '했는데', '기...","['낫', '배드', '기말고사', '서술', '문제', '비중', '제일', '점...","그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술형, 1,..."
2,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,22년 2학기 수강자,4,0,인문,13,226,"[('기', 'Modifier'), ('창', 'Noun'), ('강의', 'Nou...","['기', '창', '강의', '평', '피드백', '없고', '강의', '안도',...","['창', '강의', '평', '피드백', '강의', '안도', '꿀강', '퀴즈'...","기창, 강의평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한, 꿀강,..."
3,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...,22년 2학기 수강자,3,0,인문,13,96,"[('과제', 'Noun'), ('하나', 'Noun'), ('도', 'Josa')...","['과제', '하나', '도', '없고', '출석', '퀴즈', '2', '번', ...","['과제', '하나', '출석', '퀴즈', '번', '말', '번만', '퀴즈',...","과제, 하나, 도, 없고, 출석, 퀴즈, 2번, 기말, 1번만, 하면, 됨, 퀴즈,..."
4,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,KCU 5과목 정도 수강했는데 그중에 제일 별로 퀴즈2번 기말고사1번 수업에서 안 ...,22년 2학기 수강자,3,0,인문,13,165,"[('KCU', 'Alpha'), ('5', 'Number'), ('과목', 'No...","['KCU', '5', '과목', '정도', '수강', '했는데', '그', '중'...","['과목', '정도', '수강', '중', '제일', '별로', '퀴즈', '번',...","KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44840,영어회화,3,실습,x,2학년,전공,영문,리우빅터,회화 강사 그 이하 밑에 말하는 게 모두 사실 정말 영어과목 줄이던지 제발 영어 따...,19년 2학기 수강자,1,3,자연,4,119,"[('회화', 'Noun'), ('강사', 'Noun'), ('그', 'Noun')...","['회화', '강사', '그', '이하', '밑', '에', '말', '하는', '...","['회화', '강사', '그', '이하', '밑', '말', '게', '모두', '...","회화, 강사, 그, 이하, 밑, 에, 말, 하는, 게, 모두, 사실, 정말, 영어,..."
44841,영어회화,3,실습,x,2학년,전공,영문,리우빅터,c플 나왔습니다 시험이 무슨 2주마다 한번 있는데 준비를 해가나 안해가나 점수는 크...,19년 2학기 수강자,1,6,자연,4,169,"[('c', 'Alpha'), ('플', 'Exclamation'), ('나왔습니다...","['c', '플', '나왔습니다', '시험', '이', '무슨', '2', '주마'...","['시험', '무슨', '주마', '한번', '준비', '해', '가나', '점수'...","c, 플, 나왔습니다, 시험, 이, 무슨, 2, 주마, 다, 한번, 있는데, 준비,..."
44842,영어회화,3,실습,x,2학년,전공,영문,리우빅터,티쳐 2 그룹2 파트너2 안그래도 다른 과제 때문에 바쁜데 1학점짜리가 제일 바쁨 ...,19년 2학기 수강자,1,3,자연,4,167,"[('티', 'Noun'), ('쳐', 'Verb'), ('2', 'Number')...","['티', '쳐', '2', '그룹', '2', '파트너', '2', '안', '그...","['티', '그룹', '파트너', '다른', '과제', '때문', '학점', '리가...","티, 쳐, 2, 그룹, 2, 파트너, 2, 안, 그래도, 다른, 과제, 때문, 에,..."
44843,영어회화,3,실습,x,2학년,전공,영문,리우빅터,19년 2학기 들은사람으로서 한마디 하는데 시험은 총6번입니다 티쳐챗2회 그룹챗2회...,19년 2학기 수강자,1,1,자연,4,262,"[('19년', 'Number'), ('2', 'Number'), ('학기', 'N...","['19년', '2', '학기', '들은', '사람', '으로서', '한마디', '...","['학기', '사람', '한마디', '시험', '총', '번', '티', '챗', ...","19년, 2학기, 들은, 사람, 으로서, 한마디, 하는데, 시험, 은, 총, 6, ..."


In [126]:
df.to_csv('everytime_review_data_bitri반영.csv',index=False, encoding='utf-8-sig')

In [127]:
temp_df2

,형태소
0,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움"
1,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술형, 1,..."
2,"기창, 강의평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한, 꿀강,..."
3,"과제, 하나, 도, 없고, 출석, 퀴즈, 2번, 기말, 1번만, 하면, 됨, 퀴즈,..."
4,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."
...,...
44840,"회화, 강사, 그, 이하, 밑, 에, 말, 하는, 게, 모두, 사실, 정말, 영어,..."
44841,"c, 플, 나왔습니다, 시험, 이, 무슨, 2, 주마, 다, 한번, 있는데, 준비,..."
44842,"티, 쳐, 2, 그룹, 2, 파트너, 2, 안, 그래도, 다른, 과제, 때문, 에,..."
44843,"19년, 2학기, 들은, 사람, 으로서, 한마디, 하는데, 시험, 은, 총, 6, ..."


In [134]:
#경고제어
import warnings
warnings.simplefilter("ignore") # 일치하는 경고 인쇄 x, simplefilter : 경고 필터 명세 목록에 간단한 항목을 삽입

#텍스트 분석에 필요한 konlpy 불러오기 및 버전 확인
import konlpy
konlpy.__version__

#KoNLPy의 OKT(형태소 분석기) 불러오기
from konlpy.tag import Okt

import nltk
okt = Okt()

In [135]:
a = temp_df2['형태소'][0]

In [136]:
a

'첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움'

In [139]:
okt.nouns(a)

['첫', '강의', '퀄리티', '쉬움']

In [152]:
reviews['개정형태소'] = temp_df2['형태소']

In [153]:
reviews.head()

,리뷰,형태소,형태소_bi,형태소_tri,개정형태소
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움","첫kcu, kcu강의, 강의퀄리티, 퀄리티떨어지지만, 떨어지지만에, 에이쁠, 이쁠받...","첫kcu강의, kcu강의퀄리티, 강의퀄리티떨어지지만, 퀄리티떨어지지만에, 떨어지지만...","첫, kcu, 강의, 퀄리티, 떨어지지만, 에이쁠, 받기, 쉬움"
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형, ...","그저그래요, 그래요다른건, 다른건다, 다낫, 낫배드, 배드했는데, 했는데기말고사, ...","그저그래요다른건, 그래요다른건다, 다른건다낫, 다낫배드, 낫배드했는데, 배드했는데기...","그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술형, 1,..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,"기, 창, 강의, 평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한,...","기창, 창강의, 강의평, 평피드백, 피드백없고, 없고강의, 강의안도, 안도없지만, ...","기창강의, 창강의평, 강의평피드백, 평피드백없고, 피드백없고강의, 없고강의안도, 강...","기창, 강의평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한, 꿀강,..."
3,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...,"과제, 하나, 도, 없고, 출석, 퀴즈, 2, 번, 기, 말, 1, 번만, 하면, ...","과제하나, 하나도, 도없고, 없고출석, 출석퀴즈, 퀴즈2, 2번, 번기, 기말, 말...","과제하나도, 하나도없고, 도없고출석, 없고출석퀴즈, 출석퀴즈2, 퀴즈2번, 2번기,...","과제, 하나, 도, 없고, 출석, 퀴즈, 2번, 기말, 1번만, 하면, 됨, 퀴즈,..."
4,KCU 5과목 정도 수강했는데 그중에 제일 별로 퀴즈2번 기말고사1번 수업에서 안 ...,"KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ...","KCU5, 5과목, 과목정도, 정도수강, 수강했는데, 했는데그, 그중, 중에, 에제...","KCU5과목, 5과목정도, 과목정도수강, 정도수강했는데, 수강했는데그, 했는데그중,...","KCU, 5, 과목, 정도, 수강, 했는데, 그, 중, 에, 제일, 별로, 퀴즈, ..."


In [154]:
from collections import Counter
import collections

# bigrams = []
# for i in range(len(reviews)):
a = reviews['형태소'][0]
a = a.split(', ')

In [155]:
a

['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에', '이쁠', '받기', '쉬움']

In [157]:
b = reviews['개정형태소'][0]
b = b.split(', ')

In [158]:
b

['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에이쁠', '받기', '쉬움']

In [159]:
len(a), len(b)

(9, 8)

In [164]:
low = df[(df['평가별점수']==1) | (df['평가별점수']==2)]

In [165]:
low.to_csv('평점별리뷰/1,2점리뷰.csv', encoding='utf-8-sig', index=False)

In [166]:
low2 = df[(df['평가별점수']==1) | (df['평가별점수']==2) | (df['평가별점수']==3)]

In [167]:
low2.to_csv('평점별리뷰/1,2,3점리뷰.csv', encoding='utf-8-sig', index=False)

In [168]:
high = df[(df['평가별점수']==5)]

In [169]:
high.to_csv('평점별리뷰/5점리뷰.csv', encoding='utf-8-sig', index=False)